In [1]:
import requests, zipfile, io
import pandas as pd
import numpy as np
import bokeh.plotting as bpl
import datetime as dt

In [2]:
bpl.output_notebook()

Loading BokehJS ...

In [33]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "05/12/2020"

In [34]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Adquisición de datos históricos, esta parte solo es necesaria una vez, guarda los archivos a disco

In [35]:
url = "http://187.191.75.115/gobmx/salud/datos_abiertos/historicos/datos_abiertos_covid19_"

In [36]:
fechas_zip = fechas[:-1].strftime("%d.%m.%Y")

In [36]:
for fecha in fechas_zip:
    r = requests.get(url + fecha + ".zip")
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(path="./datos_federales/")

In [38]:
url + fechas_zip[0] + ".zip"

'http://187.191.75.115/gobmx/salud/datos_abiertos/historicos/datos_abiertos_covid19_12.04.2020.zip'

Adquisición de los últimos datos, se necesita ejecutar diario para estar actualizado, si no se ejecuta diario la adquisición histórica puede ayudar a obtener los datos que faltan

In [39]:
urlnew = "http://187.191.75.115/gobmx/salud/datos_abiertos/datos_abiertos_covid19.zip"

In [6]:
r = requests.get(urlnew)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall(path="./datos_federales/")

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [10]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [11]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [12]:
len(frames)

31

In [13]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [14]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")

In [15]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO'],
      dtype='object')

Agrupación de registros por id único

In [16]:
grupos_id = all_data.groupby("ID_REGISTRO")

Definición de dataframe por pacientes con fechas relevantes a su historia

In [17]:
pacientes = grupos_id[["FECHA_SINTOMAS","FECHA_ARCHIVO","FECHA_INGRESO","FECHA_DEF"]].min()

Definición de función para extracción de fecha de primer evento en la historia de un paciente

In [18]:
def fechador(frame,columna,valor):
    sframe = frame.sort_values("FECHA_ARCHIVO")
    fframe = sframe[sframe[columna]==valor]
    if(len(fframe)==0):
        return None
    ind_s = sframe.index[0]
    ind_f = fframe.index[0]
    if(ind_s==ind_f):
        return sframe["FECHA_INGRESO"].min()
    else:
        return fframe.loc[ind_f,"FECHA_ARCHIVO"]

Obtención de fechas de primer evento para todos los pacientes para algunos eventos importantes

In [26]:
fec_pos = grupos_id.apply(lambda x: fechador(x,"RESULTADO",1))
fec_neg = grupos_id.apply(lambda x: fechador(x,"RESULTADO",2))
fec_hos = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",1))
fec_amb = grupos_id.apply(lambda x: fechador(x,"TIPO_PACIENTE",2))
fec_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",1))
fec_noint = grupos_id.apply(lambda x: fechador(x,"INTUBADO",2))
fec_noa_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",97))
fec_ign_int = grupos_id.apply(lambda x: fechador(x,"INTUBADO",99))
fec_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",1))
fec_noneu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",2))
fec_ign_neu = grupos_id.apply(lambda x: fechador(x,"NEUMONIA",99))
fec_uci = grupos_id.apply(lambda x: fechador(x,"UCI",1))
fec_nouci = grupos_id.apply(lambda x: fechador(x,"UCI",2))
fec_noa_uci = grupos_id.apply(lambda x: fechador(x,"UCI",97))
fec_ign_uci = grupos_id.apply(lambda x: fechador(x,"UCI",99))

In [27]:
pacientes["FECHA_POSITIVO"] = fec_pos
pacientes["FECHA_NEGATIVO"] = fec_neg
pacientes["FECHA_HOSPITALIZACION"] = fec_hos
pacientes["FECHA_AMBULATORIO"] = fec_amb
pacientes["FECHA_INTUBACION"] = fec_int
pacientes["FECHA_NO_INTUBACION"] = fec_noint
pacientes["FECHA_NO_APL_INTUBACION"] = fec_noa_int
pacientes["FECHA_SE_IGN_INTUBACION"] = fec_ign_int
pacientes["FECHA_NEUMONIA"] = fec_neu
pacientes["FECHA_NO_NEUMONIA"] = fec_noneu
pacientes["FECHA_SE_IGN_NEUMONIA"] = fec_ign_neu
pacientes["FECHA_UCI"] = fec_uci
pacientes["FECHA_NO_UCI"] = fec_nouci
pacientes["FECHA_NOAPL_UCI"] = fec_noa_uci
pacientes["FECHA_SE_IGN_UCI"] = fec_ign_uci

Exportar las historias por paciente a un archivo

In [29]:
fecha_file = fechas[-1].strftime("%d_%m_%Y")
pacientes.to_csv("./datos_procesados/historia_pacientes_" + fecha_file + ".csv")

Búsqueda de pacientes en específico con alguna condición

In [28]:
columna = "FECHA_NO_UCI"
pacientes[(pacientes[columna].notna())&(pacientes["FECHA_INGRESO"]!=pacientes[columna])]

,FECHA_SINTOMAS,FECHA_ARCHIVO,FECHA_INGRESO,FECHA_DEF,FECHA_POSITIVO,FECHA_NEGATIVO,FECHA_HOSPITALIZACION,FECHA_AMBULATORIO,FECHA_INTUBACION,FECHA_NO_INTUBACION,FECHA_NO_APL_INTUBACION,FECHA_SE_IGN_INTUBACION,FECHA_NEUMONIA,FECHA_NO_NEUMONIA,FECHA_SE_IGN_NEUMONIA,FECHA_UCI,FECHA_NO_UCI,FECHA_NOAPL_UCI,FECHA_SE_IGN_UCI
ID_REGISTRO,,,,,,,,,,,,,,,,,,,
00334e,2020-04-28,2020-05-05,2020-05-04,NaT,2020-05-08,NaT,2020-05-04,2020-05-06,NaT,2020-05-06,2020-05-04,NaT,2020-05-04,NaT,NaT,NaT,2020-05-06,2020-05-04,NaT
003aca,2020-04-13,2020-04-23,2020-04-18,NaT,2020-04-18,NaT,2020-04-18,2020-04-26,NaT,2020-04-26,2020-04-18,NaT,NaT,2020-04-18,NaT,NaT,2020-04-26,2020-04-18,NaT
0041f8,2020-04-15,2020-04-16,2020-04-16,NaT,NaT,2020-04-20,2020-04-16,2020-04-17,NaT,2020-04-17,2020-04-16,NaT,NaT,2020-04-16,NaT,NaT,2020-04-17,2020-04-16,NaT
00491a,2020-04-20,2020-05-02,2020-04-30,NaT,2020-05-03,NaT,2020-04-30,2020-05-12,NaT,2020-05-12,2020-04-30,NaT,NaT,2020-04-30,NaT,NaT,2020-05-12,2020-04-30,NaT
00495f,2020-04-05,2020-04-12,2020-04-05,NaT,2020-04-13,NaT,2020-04-05,2020-04-15,NaT,2020-04-15,2020-04-05,NaT,NaT,2020-04-05,NaT,NaT,2020-04-15,2020-04-05,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1e2bc0,2020-04-19,2020-04-26,2020-04-26,NaT,NaT,2020-04-30,2020-04-26,2020-04-27,NaT,2020-04-27,2020-04-26,NaT,NaT,2020-04-26,NaT,NaT,2020-04-27,2020-04-26,NaT
1e2ef0,2020-04-22,2020-04-27,2020-04-26,NaT,NaT,2020-04-29,NaT,2020-04-26,NaT,2020-04-26,NaT,NaT,2020-04-26,2020-05-05,NaT,2020-04-26,2020-05-05,NaT,NaT
1e5a17,2020-04-15,2020-04-27,2020-04-15,NaT,NaT,NaT,2020-04-15,2020-05-04,NaT,2020-05-04,2020-04-15,NaT,NaT,2020-04-15,NaT,NaT,2020-05-04,2020-04-15,NaT


In [19]:
caso_tmp = all_data[all_data["ID_REGISTRO"]=="004d53"][["ID_REGISTRO","RESULTADO","TIPO_PACIENTE","FECHA_ARCHIVO","FECHA_SINTOMAS","FECHA_INGRESO","FECHA_DEF","INTUBADO","NEUMONIA","UCI"]]
caso_tmp

,ID_REGISTRO,RESULTADO,TIPO_PACIENTE,FECHA_ARCHIVO,FECHA_SINTOMAS,FECHA_INGRESO,FECHA_DEF,INTUBADO,NEUMONIA,UCI
655686,004d53,2,1,2020-04-25,2020-04-17,2020-04-19,NaT,97,2,97
722814,004d53,2,1,2020-04-26,2020-04-17,2020-04-19,NaT,97,2,97
792336,004d53,2,1,2020-04-27,2020-04-17,2020-04-19,NaT,97,2,97
887097,004d53,2,1,2020-04-28,2020-04-17,2020-04-19,NaT,97,2,97
944266,004d53,2,1,2020-04-29,2020-04-17,2020-04-19,NaT,97,2,97
1028338,004d53,2,2,2020-04-30,2020-04-19,2020-04-29,NaT,2,2,2
1115969,004d53,2,2,2020-05-01,2020-04-19,2020-04-29,NaT,2,2,2
1221922,004d53,2,2,2020-05-02,2020-04-19,2020-04-29,NaT,2,2,2
1309359,004d53,2,2,2020-05-03,2020-04-19,2020-04-29,NaT,2,2,2
1369812,004d53,1,2,2020-05-04,2020-04-19,2020-04-29,NaT,2,2,2
